In [145]:
from pyecharts import options as opts
from pyecharts.charts import Graph
sys.path.append("/opt/miniconda3/lib/python3.7/site-packages") 
import pandas as pd
import numpy as np
import os


In [146]:
from py2neo import Node, Relationship,NodeMatcher,RelationshipMatcher
import py2neo
graph = py2neo.Graph(
    "http://localhost:7474", 
    username="neo4j", 
    password="123456"
)

In [127]:

matcher_node = NodeMatcher(graph)
matcher_relation = RelationshipMatcher(graph)
node = matcher_node.match("brand")
# relation = matcher_relation.matcher(r_type="Cited").limit(50)
list(node)

[Node('brand', name='星巴克', subtype='name')]

In [157]:
cypher_1 = "MATCH (n) RETURN id(n) as id, n.name,n.subtype as subtype, labels(n) as label LIMIT 500"
cypher_2 = "MATCH (a)-[r]->(b) RETURN id(a) as a_id,a.name, b.name, type(r), id(b) as b_id LIMIT 1000"

nodes = graph.run(cypher_1 ).data()
links = graph.run(cypher_2 ).data()

nodes = pd.DataFrame(nodes)
links = pd.DataFrame(links)



In [158]:
nodes_data = []
links_data = []
categories = []
# 新建包含节点类别的字典，方便查询获得类别编号
cateName = {'brand':0, 'customer':1, 'effect':2,'feature':3, 'identification':4, 'platform':5, 'product':6, 'project':7}

# 添加结点
for i in range(0,len(nodes)):
    name = nodes.iloc[i,1]
    cate = cateName[nodes.iloc[i,3][0]]
    nodes_data.append({"name":name,"symbolSize":10,"category":cate})

# 添加关系
for i in range(0,len(links)):
    nameA = links.iloc[i,1]
    nameB = links.iloc[i,2]
    typer = links.iloc[i,3]
    links_data.append({"source":nameA,"target":nameB,"value":typer})
  
# 添加类别
categories.append({"name":"brand"})
categories.append({"name":"customer"})
categories.append({"name":"effect"})
categories.append({"name":"feature"})
categories.append({"name":"identification"})
categories.append({"name":"platform"})
categories.append({"name":"product"})
categories.append({"name":"project"})

c = (
    Graph(init_opts=opts.InitOpts(width="900px", height="900px"))
    .add(
        "",
        nodes_data,
        links_data,
        categories,
        repulsion=50, # 排斥力
        edge_label=opts.LabelOpts(
            is_show=True, position="middle", formatter="{c}"),
        # is_rotate_label=True,
        # label_opts=opts.LabelOpts(position="right"),
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(title="brand knowledge graph"),
        legend_opts=opts.LegendOpts(orient="vertical", pos_left="2%", pos_top="20%"),
    )
    .render("graph_with_edge_options.html")
)


